In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold

In [2]:
train_lgbm = pd.read_csv('train_lgbmcv.csv')
train_rfcv = pd.read_csv('train_rfcv.csv')
train = pd.read_csv('train.csv')

In [4]:
train = train[['card_id', 'target']]

In [7]:
predictions_rfcv = pd.read_csv('test_rfcv.csv')
predictions_lgbm = pd.read_csv('test_lgbmcv.csv')

In [15]:
train_lgbm.shape, train_rfcv.shape, train.shape, predictions_lgbm.shape, predictions_rfcv.shape

((201917, 1), (201917, 1), (201917, 2), (123623, 1), (123623, 1))

In [19]:
def stack_model(oof_list, prediction_list, y):
    
    train_stack = np.hstack(oof_list)
    test_stack = np.hstack(prediction_list)
    
    oof = np.zeros(train_stack.shape[0])
    predictions = np.zeros(test_stack.shape[0])
    
    from sklearn.model_selection import RepeatedKFold
    folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=99)
    
    for n, (trn_idx, val_idx) in enumerate(folds.split(train_stack, y)):
        
        print(f'Fold No{n+1}')
        
        trn_data, trn_y = train_stack[trn_idx], y[trn_idx]
        val_data, val_y = train_stack[val_idx], y[val_idx]
        
        print('-'*10 + 'Stacking' +'-'*10)
        
        from sklearn.linear_model import BayesianRidge
        clf = BayesianRidge()
        clf.fit(trn_data, trn_y)
        oof[val_idx] = clf.predict(val_data)
        predictions += clf.predict(test_stack)/(5*2)
        
    return predictions

In [13]:
target = train['target'].values

In [16]:
target.shape

(201917,)

In [20]:
predistion_stack = stack_model([train_rfcv, train_lgbm],[predictions_rfcv, predictions_lgbm], target)

Fold No1
----------Stacking----------
Fold No2
----------Stacking----------
Fold No3
----------Stacking----------
Fold No4
----------Stacking----------
Fold No5
----------Stacking----------
Fold No6
----------Stacking----------
Fold No7
----------Stacking----------
Fold No8
----------Stacking----------
Fold No9
----------Stacking----------
Fold No10
----------Stacking----------


In [21]:
predistion_stack.shape

(123623,)

In [22]:
test_stack = pd.read_csv('submission_rfcv5.csv')

In [23]:
test_stack['target'] = predistion_stack

In [25]:
test_stack.to_csv('submission_stacking.csv', index = False)